In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
data_path = '../../data'

cols1 = ['round_number', 'block_number', 'sequence_number', 'block_type', 'winning_probability',
'aaron_mood', 'probability_aaron_show', 'reward_value', 'reaction_time', 'choice',
'status_of_play', 'outcome', 'accumulated_outcomes', 'age', 'gender', 'major', 
'MCQ_Q1', 'MCQ_Q2', 'MCQ_Q3', 'MCQ_Q4', 'MCQ_Q5', 'MCQ_Q6'] 

cols2 = ['pre_game_strategy', 'wager', 'post_game_quiz_q1', 'post_game_quiz_q2', 'post_game_quiz_q3',
'post_game_quiz_q3_other', 'post_game_quiz_correct', 'odds_guess_one_reply', 'odds_guess_one_did_win',
'odds_guess_two_reply', 'odds_guess_two_did_win', 'odds_guess_three_reply', 'odds_guess_three_did_win',
'filename', 'id', 'treatment', 'craver']

cols_full = cols1 + ['chance_guess'] + cols2

data_full = pd.DataFrame(columns=cols_full)

In [3]:
ID = 1
missed = []
trials = []

for treat in ['control', 'test']:
    treat_path = os.path.join(data_path, treat)
    for sub in [f for f in os.listdir(treat_path) if 'xls' in f]:
        file_path = os.path.join(treat_path, sub)

        temp = pd.read_excel(file_path, header = 0)

        if temp.shape[0] < 630:
            print(f'{sub}, {sum(temp["choice"] == 2)}, {temp.shape}')

        if 'T' not in sub and sum(temp['choice'] == 2) < 7:
            temp['filename'] = sub
            temp['id'] = ID
            temp['treatment'] = treat

            if temp.loc[0, 'wager'] != 0:
                temp['craver'] = 0
            else:
                temp['craver'] = 1

            if temp.shape[1] == 38:
                chance_guess = pd.Series([np.nan]*len(temp))
                temp = pd.concat([temp[cols1], chance_guess, temp[cols2]], axis=1)
                temp.columns = cols_full
            if 'MCQ_Q4' not in temp.columns:
                mcq_data = pd.DataFrame({'MCQ_Q4': [1]*len(temp),
                                         'MCQ_Q5': [1]*len(temp),
                                         'MCQ_Q6': [1]*len(temp)})
                cols1_mcq = [item for item in cols1 if item not in ['MCQ_Q4', 'MCQ_Q5', 'MCQ_Q6']]
                cols1_mcq.append('chance_guess')
                temp = pd.concat([temp[cols1_mcq], mcq_data, temp[cols2]], axis=1)
                temp.columns = cols_full

            data_full = pd.concat([data_full, temp])
            ID += 1
        else:
            print(f'{sub}, {sum(temp["choice"] == 2)}, {temp.shape}')

katie.ringdahl2002@gmail.com_control_id-228.xls, 0, (455, 33)


In [4]:
data_full.shape

(125195, 40)

In [5]:
data_full.to_csv(
    os.path.join(data_path, 'data_full.csv'), index=False)